In [9]:
# probelm need to be regenerated
import pandas as pd
prefix = '14-9F-3C-DA-5B-26'
output_path = '/Volumes/hex/ST-1441993385-res/'
p_385 = pd.read_csv(output_path + 'ST-1441993385_2.0.csv')

p_385.columns
len(p_385.index)
# 207799

207799

In [6]:
from matplotlib import pyplot
%matplotlib inline
xaxis = p_385[['time']]
yaxis = p_385[['phq_1']]
yaxis2 = p_385[['phq_2']]
fig=pyplot.figure(figsize=(18, 5), dpi= 80, facecolor='w', edgecolor='k')
pyplot.plot(xaxis, yaxis, color='red', label='phq_1')
pyplot.plot(xaxis, yaxis2, color='blue', label='phq_2')

KeyError: "None of [Index(['phq_1'], dtype='object')] are in the [columns]"

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [10]:
def display_scores(score):
    print('scores:', score)
    print('Mean:', score.mean())
    print('std dev:', score.std())

In [7]:
train_set, test_set = train_test_split(p_385, test_size=0.25, random_state=42, shuffle=True)

In [8]:
X_train = train_set.loc[:,'accX':'work_time']
y_train_1 = train_set.loc[:, ['phq_1']].values.ravel()
y_train_2 = train_set.loc[:, ['phq_2']].values.ravel()

X_test = test_set.loc[:,'accX':'work_time']
y_test_1 = test_set[['phq_1']].values.ravel()
y_test_2 = test_set[['phq_2']].values.ravel()

In [9]:
X_train.head()

,accX,accX_std,accY,accY_std,accZ,accZ_std,gyrX,gyrX_std,gyrY,gyrY_std,...,alc,mood,tense,tired,period,rumination,socialize,socialize_val,sport_time,work_time
92742,0.701401,0.360205,0.272681,0.203608,0.449268,0.203608,0.292532,0.134348,0.359708,0.101257,...,0.0,0.875,0.500,0.9,0,1.0,0.173469,0.651685,0.0,0.000000
135128,0.400752,0.233525,0.288393,0.432990,0.541646,0.432990,0.289270,0.055091,0.356654,0.033470,...,0.0,1.000,0.875,0.8,0,1.0,0.173469,0.820225,0.0,0.000000
74545,0.702767,0.002559,0.085656,0.002577,0.623423,0.002577,0.289458,0.001779,0.357579,0.000733,...,0.0,0.750,0.875,0.2,0,0.8,0.112245,0.764045,0.0,0.000000
145738,0.453707,0.311580,0.161683,0.181057,0.538617,0.181057,0.287171,0.249536,0.375532,0.188710,...,0.0,0.875,0.875,0.8,0,1.0,0.204082,0.853933,0.0,0.000000
113282,0.545268,0.001280,0.502281,0.001289,0.997981,0.001289,0.289570,0.000102,0.356931,0.000154,...,0.0,0.625,0.250,0.0,0,1.0,0.806122,0.269663,0.0,0.717391


In [25]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=35, n_jobs=-1 ,oob_score=True)
rnd_reg.fit(X_train, y_train_1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=35,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [26]:
y_pred_1 = rnd_reg.predict(X_train)

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np
forest_mse = mean_squared_error(y_train_1, y_pred_1)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1.2397195376250258

In [67]:
from sklearn.utils import shuffle
y_shuffled = shuffle(y_pred_1)
forest_mse_shuffled = mean_squared_error(y_train_1, y_shuffled)
forest_rmse_shuffled = np.sqrt(forest_mse_shuffled)
forest_rmse_shuffled

2.7900542055076825

In [28]:
forest_scores = cross_val_score(rnd_reg, X_train, y_train_1,scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

scores: [1.25087301 1.23712307 1.23322317 1.24298394 1.23310135 1.24879575
 1.25023498 1.21127483 1.23812528 1.23860825]
Mean: 1.2384343612678923
std dev: 0.011032291218734152


In [30]:
# on test set
from sklearn.metrics import r2_score

y_pred_test = rnd_reg.predict(X_test)
r2_score(y_test_1, y_pred_test)

0.6802747327477925